In [1]:
from tensorflow.keras.applications import vgg16
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
model = vgg16.VGG16()

553467096/553467096 [==============================] - 54s 0us/step


In [3]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [6]:
batch_size = 32   # 한셋트에 몇개의 자료를 넣을것인가? 기본 32로 많이하나 조정해도됨. 숫자를 적게할수록 처리속도가 느려짐
img_height = 180
img_width = 180

data_dir='c:/data/flower_photos/'
# 전체 3670개의 이미지자료에서 80%의 2936개를 트레이닝(훈련)자료로 세팅함
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,   # 3670*80%=2936, subset이 training여서 1-0.2임
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)     # 2936개의이미지를 32개씩 세트해서 묶음. 즉 92개세트가 나옴 (92세트*32)

# 전체 3670개의 이미지자료에서 20%의 734개를 테스트데이터로 세팅함
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# train_ds에 할당된 data_dir 폴더명
class_names = train_ds.class_names
print(class_names)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [7]:
data_augmentation = tf.keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

# 레이어층에서 자동으로 정규화까지 해줌
preprocess_input = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [8]:
base_model = VGG16(input_shape=(180,180,3),
                   include_top=False,
                   weights='imagenet')
base_model.trainable = False # 기존 vgg에서 제공하는 imgnet의 w,b를 사용

global_average_layer = tf.keras.layers.GlobalAveragePooling2D() # Flatten() 역할
prediction_layer = tf.keras.layers.Dense(5,activation='softmax')

58889256/58889256 [==============================] - 3s 0us/step


In [9]:
inputs = tf.keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs =prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 5, 5, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                             

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # y값이 원핫인코딩이 안되어 있어서임
              metrics=['accuracy'])


# 모델 최적화를 위한 설정 구간입니다.

modelpath="./models/전이학습vgg16.hdf5"

checkpointer = ModelCheckpoint(filepath=modelpath, 
                               monitor='val_loss',       # val_loss값을 기준으로
                               verbose=1,                # 실행결과를 화면에 출력함. verbose=0하면 실행결과가 화면에 나타나지않음
                               save_best_only=True)  # 가장좋은(즉 loss숫자가 가장 낮은) 모델을 저장함

early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                        patience=5) # 실행하다가 5번이상 더 좋은 결과가 없으면 중단

epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[early_stopping_callback,checkpointer]
)

Epoch 1/100
92/92 [==============================] - ETA: 0s - loss: 1.5166 - accuracy: 0.3638
Epoch 1: val_loss improved from inf to 1.29183, saving model to ./models\전이학습vgg16.hdf5
92/92 [==============================] - 331s 4s/step - loss: 1.5166 - accuracy: 0.3638 - val_loss: 1.2918 - val_accuracy: 0.5899
Epoch 2/100
92/92 [==============================] - ETA: 0s - loss: 1.2277 - accuracy: 0.5725
Epoch 2: val_loss improved from 1.29183 to 1.10058, saving model to ./models\전이학습vgg16.hdf5
92/92 [==============================] - 327s 4s/step - loss: 1.2277 - accuracy: 0.5725 - val_loss: 1.1006 - val_accuracy: 0.6662
Epoch 3/100
92/92 [==============================] - ETA: 0s - loss: 1.0706 - accuracy: 0.6393
Epoch 3: val_loss improved from 1.10058 to 0.99337, saving model to ./models\전이학습vgg16.hdf5
92/92 [==============================] - 324s 4s/step - loss: 1.0706 - accuracy: 0.6393 - val_loss: 0.9934 - val_accuracy: 0.6962
Epoch 4/100
92/92 [==============================] - 

KeyboardInterrupt: 